<a href="https://colab.research.google.com/github/ConorDataHub/AI/blob/main/TestAStarAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A * Search Algorithm

- This algorithm is know as an informed search algorithm, meaning that it has a "heuristic" or help in finding its goal state

In [9]:
# The below is a an array which contains the starting point or starting state of the algorithm. 
# The 0 is representative of a blank tile in an N-Puzzle
# This is always the the number that will be moved to try and reach the goal state


initial_state = [0,  3,  5,  7,
                 1, 11, 13, 15,
                 9,  4,  6,  8,
                 2, 10, 12, 14]

In [10]:
# This array is the goal state
# The goal of the algorithm is for the initial state to reach the goal state in the shortest number of moves

goal_state = [1,  3,  5,  5,
              9, 11, 13, 15,
              2,  4,  6,  8,
              10, 12, 14, 0]

In [11]:
# This is the size of the list (determined by counting the numbers in the initial_state plus 1)
size = 16  
# Dimension is the squared root of the size - sqr(16)=4
dimension = 4             

In [12]:
# This function takes a state as an array 
# Its purpose is to keep the score of the number of tiles that are out of place at each stage of the algorithm
# E.g., it will compare index[i] in the initial_state against index[i] of the goal_state and if they are not equal it will count 1 else it won't count anything


def check_diff_score(state):        
  diff = 0
  for i in range(size):
      current_item = state[i]
      if current_item != 0 and current_item != goal_state[i]:
          diff +=1
  return diff


In [16]:
print("Initial State: ", initial_state)
print("Goal State:    ", goal_state)
diff = check_diff_score(initial_state)
print("Numbers out of place :", diff)

Initial State:  [0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14]
Goal State:     [1, 3, 5, 5, 9, 11, 13, 15, 2, 4, 6, 8, 10, 12, 14, 0]
Numbers out of place : 7


In [40]:
# Function to print states as a 4x4 matrix

def print_state(state):               
  for i in range(size):              
    if i % dimension == 0:
      print("")                      # This gives us a new line if index = 4 (count starts from 0)
    print(str(state[i]), "", end="")      # Prints the current value of the state as a string
  print("\n")


In [44]:
# Example of the format that is printed
print_state(initial_state) 




0 3 5 7 
1 11 13 15 
9 4 6 8 
2 10 12 14 



In [45]:
# These dictionaries will have the list of items inside of it 
# Every value has a key associated with it. 
# The hashing helps to reduce time complexity compared to lists O(1) v O(n)

# Map the index of a 2 dimensional array to a 1 dimensional index 
matrix_to_1D_mapping = {} 

# Map the index of a 1 dimensional array to a 2 dimensional index         
index_to_2D_mapping = {}                                  

In [46]:
# This function will give us dictionary values to search the array by its 1D and 2D index

index = 0
for row in range(dimension):
  for column in range(dimension):
    matrix_to_1D_mapping[(row, column)] = index
    index_to_2D_mapping[index] = (row, column)
    index += 1


In [47]:
print(matrix_to_1D_mapping)
print(index_to_2D_mapping)

{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (1, 0): 4, (1, 1): 5, (1, 2): 6, (1, 3): 7, (2, 0): 8, (2, 1): 9, (2, 2): 10, (2, 3): 11, (3, 0): 12, (3, 1): 13, (3, 2): 14, (3, 3): 15}
{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (1, 0), 5: (1, 1), 6: (1, 2), 7: (1, 3), 8: (2, 0), 9: (2, 1), 10: (2, 2), 11: (2, 3), 12: (3, 0), 13: (3, 1), 14: (3, 2), 15: (3, 3)}


In [61]:
#Function to change list to string
def get_state_id(state):
  string_list = []
  for item in state:
    string_list.append(str(item))
  return ", ".join(str(v) for v in state)               # For every item inside the list - cast that item to a string and join with a comma - this is so we can use a string to query the dictionary

In [63]:
print("List of ints:")
print(initial_state)
id = get_state_id(initial_state)
print("Casted to String:")
print(id)

List of ints:
[0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14]
Casted to String:
0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14


In [65]:
def get_node_state(id):
  state = []
  split_list = id.split(",")
  for item in split_list:
    state.append(int(item))
  print(split_list)
  return state

In [67]:
get_node_state("0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14")

['0', ' 3', ' 5', ' 7', ' 1', ' 11', ' 13', ' 15', ' 9', ' 4', ' 6', ' 8', ' 2', ' 10', ' 12', ' 14']


[0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14]

In [69]:
# Need two lists/dictionaries, one for open nodes and one for closed nodes(nodes of previous states)

opened_nodes = {}
closed_nodes = {}

# Current state is assigned the value of the initial state. The .copy method is used so that the original list will remain unchanged
current_state = initial_state.copy()
# Depth tracks the depths of the nodes, starting from 0
depth = 0

# This method counts the number of tiles currently out of place
initial_score = check_diff_score(current_state)
print("Initial score: ", initial_score)

if(initial_score == 0):
  print("Puzzle solved")
else:
  pass


Initial score:  7
